In [1]:
#Importing Necessary Libraries

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import plotly.express as px

# Libraries to help with data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import requests

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans


In [2]:
#Reading data
df = pd.read_csv("Medical_Examiner_Case_Archive.csv", low_memory= False)
df = df.rename(columns = {'Manner of Death' : 'Manner_of_Death', 'Primary Cause' : 'Primary_Cause','Incident Zip Code' : 'Incident_Zip', 
                                    'Residence Zip' : 'Residence_Zip', 'Residence City' : 'Residence_City', 
                                    'Chicago Ward' : 'Chicago_Ward', 'Chicago Community Area' : 'Chi_Community_Area', 'Incident City': 'Incident_City',
                                    'Commissioner District' : 'Commissioner_District', 'Primary Cause Line A' : 'Primary_Cause_A', 
                                    'Gun Related' : 'Gun_Related', 'Date of Incident' : 'Incident_Date', 'Date of Death' : 'Death_Date',
                                    'Secondary Cause' : 'Secondary_Cause', 'Opioid Related':'Opioid_Related'})

df.head()
#Filtering Data to only look at Drug Related Deaths
df_filtered = df[(df['COVID Related'] == False) & (df['Opioid_Related'] == True)]
print('Number of Rows:',len(df_filtered))


df_filtered = df_filtered.drop(columns=["COVID Related", "OBJECTID", "Residence_City", "Residence_Zip", "Primary Cause Line C"])


Number of Rows: 14083


In [3]:
poverty_df = pd.read_excel("illinois_income_by_zipcode.xlsx", usecols = [0,1,2], skiprows=range(6), header=None)

In [4]:
poverty_df = poverty_df.iloc[:-1*(9868-9846)]
poverty_df.columns = ["Incident_Zip","Income_Range", "People"]
poverty_df_copy = poverty_df.copy()


In [5]:
# Reset the index to ensure it is continuous
poverty_df_copy = poverty_df_copy.reset_index(drop=True)
# Drop every 8th row
poverty_df_copy = poverty_df_copy.drop(poverty_df.index[7::8])
poverty_df_copy = poverty_df_copy.reset_index(drop=True)

In [6]:
poverty_df_copy["Total_Per_Zip"] = poverty_df_copy["People"].iloc[::7].repeat(7).reset_index(drop=True)
poverty_df_copy["Total_Per_Zip"] = poverty_df_copy["Total_Per_Zip"].reindex(poverty_df_copy.index).ffill()

poverty_df_copy['People'] = pd.to_numeric(poverty_df_copy['People'], errors='coerce')
poverty_df_copy['Total_Per_Zip'] = pd.to_numeric(poverty_df_copy['Total_Per_Zip'], errors='coerce')
poverty_df_copy["Proportion_Income"] = poverty_df_copy["People"] / poverty_df_copy["Total_Per_Zip"]

C:\Users\bdscu\AppData\Local\Temp\ipykernel_23976\4033331605.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  poverty_df_copy["Total_Per_Zip"] = poverty_df_copy["Total_Per_Zip"].reindex(poverty_df_copy.index).ffill()


In [7]:
Proportion_Under_25000 = poverty_df_copy.iloc[8::7][["Incident_Zip","Proportion_Income"]]
Proportion_Under_25000["Incident_Zip"] = Proportion_Under_25000["Incident_Zip"].astype(int, errors='raise')  # Raise an error if conversion fails
df_filtered["Incident_Zip"] = df_filtered["Incident_Zip"].fillna(0)
df_filtered["Incident_Zip"] = df_filtered["Incident_Zip"].astype(int, errors='raise')  # Raise an error if conversion fails

In [8]:
merged_under_25000 = pd.merge(df_filtered, Proportion_Under_25000, how="inner", on="Incident_Zip")


In [9]:
under_25000 = merged_under_25000[["Incident_Zip", "Proportion_Income"]]
# under_25000 = under_25000.set_index("Incident_Zip")
under_25000.dropna(inplace=True)

C:\Users\bdscu\AppData\Local\Temp\ipykernel_23976\2889956854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  under_25000.dropna(inplace=True)


In [10]:
poverty_df_copy

,Incident_Zip,Income_Range,People,Total_Per_Zip,Proportion_Income
0,0,Total,6097080.0,6097080,1.000000
1,0,"$1 under $25,000",1917000.0,6097080,0.314413
2,0,"$25,000 under $50,000",1414080.0,6097080,0.231927
3,0,"$50,000 under $75,000",879300.0,6097080,0.144217
4,0,"$75,000 under $100,000",583160.0,6097080,0.095646
...,...,...,...,...,...
8612,99999,"$25,000 under $50,000",9300.0,35320,0.263307
8613,99999,"$50,000 under $75,000",5680.0,35320,0.160815
8614,99999,"$75,000 under $100,000",3500.0,35320,0.099094
8615,99999,"$100,000 under $200,000",3970.0,35320,0.112401


In [11]:
zips = df_filtered["Incident_Zip"].unique().tolist()[1:-2]

In [12]:
60438 in zips

True

In [13]:
url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/refs/heads/master/il_illinois_zip_codes_geo.min.json"
data = requests.get(url).json()
import json

# Filter features for Chicago ZIP codes (e.g., those starting with "606")
chicago_features = [
    feature for feature in data['features']
    if feature['properties']['ZCTA5CE10'] in zips
]

# Create a new GeoJSON object containing only Chicago ZIP code areas
chicago_geojson = {
    "type": "FeatureCollection",
    "features": chicago_features
}

# Save the filtered GeoJSON
with open('chicago_zipcodes.geojson', 'w') as f:
    json.dump(chicago_geojson, f)


In [14]:
# url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/refs/heads/master/il_illinois_zip_codes_geo.min.json"
# geojson_data = requests.get(url).json()
# # Create the Folium map

import json

with open('chicago_zipcodes.geojson', 'r') as f:
    chicago_data = json.load(f)

m = folium.Map(location=[41.8781, -87.6298], zoom_start=9)  # Adjust location to center on Illinois

# "il_illinois_zip_codes_geo.min.json"
# Add a Choropleth layer
folium.Choropleth(
    geo_data=chicago_data,  # Path to the GeoJSON file
    data=under_25000,  # DataFrame containing your data
    columns=["Incident_Zip", "Proportion_Income"],  # Columns in the DataFrame
    key_on="feature.properties.ZCTA5CE10",  # Match the GeoJSON property
    fill_color="YlOrRd",  # Choose a color scale
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Proportion in Poverty",
    bins = [0.0, 0.1,0.2,0.3,0.4,0.5,1],
).add_to(m)

In [15]:
m


In [ ]:
print(merged_under_25000["Proportion_Income"].min())
print(merged_under_25000["Proportion_Income"].max())
merged_under_25000["Proportion_Income"].sort_values(ascending=False)


In [ ]:

# Initialize a MarkerCluster
marker_cluster = MarkerCluster(name='Drug Overdoses').add_to(m)

# Iterate through the overdose data and add markers
for idx, row in merged_under_25000.dropna(subset=['latitude','longitude','Proportion_Income'], inplace=False).iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=f"ZIP: {row['Incident_Zip']}, Age: {row['Age']}, Cause: {row['Primary_Cause']}"
    ).add_to(marker_cluster)

m.save('poverty_drug_overdose.html')


In [ ]:
from folium.features import DivIcon

# Initialize a MarkerCluster
marker_cluster = MarkerCluster(name='Drug Overdoses').add_to(m)

# Iterate through the overdose data
for idx, row in merged_under_25000.dropna(subset=['latitude', 'longitude', 'Proportion_Income'], inplace=False).iterrows():
    # Add a circle marker
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=f"ZIP: {row['Incident_Zip']}, Age: {row['Age']}, Cause: {row['Primary_Cause']}"
    ).add_to(marker_cluster)

grouped = merged_under_25000.dropna(subset=['latitude', 'longitude']) \
                           .groupby('Incident_Zip', as_index=False) \
                           .agg({'latitude':'mean', 'longitude':'mean'})

# Add a marker and text label for each unique ZIP
for idx, row in grouped.iterrows():
    # Add static text label once per ZIP
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; background-color: white; border: 1px solid black; padding: 2px;">{row["Incident_Zip"]}</div>'
        )
    ).add_to(marker_cluster)

m.save('poverty_drug_overdose.html')


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [ ]:
# top_causes = df.Primary_Cause.value_counts().head(50).index.to_list()[:30]
# top_causes = df[df["Primary_Cause"].isin(top_causes)]
# top_causes = top_causes.dropna()

# top_causes = top_causes.applymap(lambda x: 1 if x is True else (0 if x is False else x))


# features = top_causes[["longitude","latitude", "Age", "Gender", "Race", "Latino", "Primary_Cause"]]
# features = pd.get_dummies(features, columns = ["Gender", "Race", "Latino", "Primary_Cause"])
# # features = features.drop(columns=["Gender", "Race", "Latino", "Primary_Cause"])

# target = top_causes["Opioid_Related"]



In [ ]:
# Step 1: Get the top causes
top_causes_list = df.Primary_Cause.value_counts().head(50).index.to_list()[:30]

# Step 2: Filter the DataFrame
top_causes = df[df["Primary_Cause"].isin(top_causes_list)]

# Step 3: Drop rows with missing values in specific columns
# (Assuming 'Opioid_Related' is your target variable)
top_causes = top_causes.dropna(subset=["longitude", "latitude", "Incident_Zip", "Age", "Gender", "Race", "Latino", "Primary_Cause", "Opioid_Related"])

# Step 4: Identify boolean columns (excluding 'Opioid_Related')
boolean_columns = top_causes.select_dtypes(include='bool').columns.tolist()
# boolean_columns = [col for col in boolean_columns if col != 'Opioid_Related']

# Step 5: Convert boolean columns to integers
top_causes[boolean_columns] = top_causes[boolean_columns].astype(int)

# Step 6: Prepare features
features = top_causes[["longitude", "latitude", "Age", "Gender", "Race", "Latino", "Primary_Cause"]]

# Step 7: One-hot encode categorical features
features = pd.get_dummies(features, columns=["Gender", "Race", "Latino", "Primary_Cause"])

# Step 8: Extract the target variable
target= top_causes["Opioid_Related"].astype(int)

# Verify target distribution
print("Target distribution:\n", target.value_counts())


In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state=42
)
# Fit multinomial logistic regression with L1 regularization (Lasso)
lasso_logistic = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial')
lasso_logistic.fit(X_train, y_train)

In [ ]:
features_weights = []
for feature, coeff in zip(features.columns,lasso_logistic.coef_[0]):
    # {key:value for key, value in zip(features.columns,lasso_logistic.coef_[0])}
    if abs(coeff) > 0.05:
        print(feature, coeff)
        